# Initial Setup:

In [ ]:
import pickle
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Data File Location

df = "/content/drive/My Drive/IE434 Deep Dive 11 project/Deep Dive 3 (Milestone 2)"
filename = '/content/drive/My Drive/IE434 Deep Dive 11 project/Deep Dive 3 (Milestone 2)/working_df.pkl'


In [ ]:
# Loading the Dataset

with open(filename, 'rb') as file:
    working_df = pickle.load(file)

# Print the first few entries of the DataFrame

print(working_df.head())

FileNotFoundError: ignored

# Handling Missing Data:

The output below presents the initial number of rows in the dataset, followed by a detailed account of null values in each column.

Initially, we had a total of 1,446,166 rows before any data cleansing. Upon inspection of individual columns, we observed that most columns have no missing data. Notable exceptions include `start_station_name`, `start_station_id`, `end_station_name`, `end_station_id`, `end_lat`, and `end_lng`, with a small percentage of missing values ranging from 0.01% to 0.34%.

Given the low percentage of missing values relative to the size of the dataset, we decided to proceed with dropping rows with any null values. This decision is under the assumption that the loss of data is minimal and that maintaining a complete case analysis is critical for our subsequent analysis.

After dropping these rows, we are left with a total of 1,441,178 rows. The impact of this operation was the removal of approximately 0.03% of the data, indicating a negligible reduction in dataset size while potentially improving data quality.

Here is the summarized information:
- Total rows before dropping null values: 1,446,166
- Total rows after dropping null values: 1,441,178

Below is the detailed breakdown of missing values before the operation:

In [ ]:
# Calculate total rows
total_rows = len(working_df)
print(f"--- Total rows before dropping null values: {total_rows}\n")

# Calculate number and percentage of null values for each column
print("Percentage and number of null values in each column: \n")
null_counts = working_df.isnull().sum()
for column, count in null_counts.items():
    percentage = (count / total_rows) * 100
    print(f"({percentage:.2f}%) : {count} null values in '{column}' ")
print(" ")

# Drop rows containing any null value
working_df.dropna(inplace=True)

# Print total number of rows after dropping
print(f"--- Total rows after dropping null values: {len(working_df)}")


Dropping rows with missing values can sometimes lead to biases if the missing data is not randomly distributed. In this case, the missing data constituted a very small proportion of the total, suggesting that the impact on the overall dataset might be limited.


In [ ]:
working_df.dropna()

# 1441178 rows × 13 columns

# Calculating Average Coordinates for Stations

To facilitate any geospatial analysis that might be required later on, we calculate the average latitude and longitude for each unique station in our dataset. This involves processing both the starting points and the destinations of the rides.

## Start Stations
First, we group the data by the `start_station_name` and calculate the mean latitude and longitude, creating a new DataFrame with these average coordinates.

## End Stations
Similarly, we repeat the process for `end_station_name` to get the average coordinates for all ending stations.

## Combining Start and End Stations
Since some stations serve as both start and end points, we combine the two sets of average coordinates into a single DataFrame. This helps us in deduplicating the stations and ensures that each unique station is represented only once.

## Final Station Coordinates
Lastly, we group by the station names again to ensure that if a station appears in both the start and end lists, we calculate the overall mean latitude and longitude for that station.


In [ ]:
#calculating avg of latitude and longitude for each of the unique start stations
start_station_avg_coords = working_df.groupby('start_station_name').agg({'start_lat': 'mean', 'start_lng': 'mean'}).reset_index()

start_station_avg_coords.rename(columns={'start_station_name':'station_name','start_lat': 'avg_latitude', 'start_lng': 'avg_longitude'}, inplace=True)

#calculating avg of latitude and longitude for each of the unique end stations
end_station_avg_coords = working_df.groupby('end_station_name').agg({'end_lat': 'mean', 'end_lng': 'mean'}).reset_index()

end_station_avg_coords.rename(columns={'end_station_name':'station_name','end_lat': 'avg_latitude', 'end_lng': 'avg_longitude'}, inplace=True)

#combining the start and end stations to find the unique number of stations with its avg lat and long.
combined_stations = pd.concat([start_station_avg_coords, end_station_avg_coords])

# Group by station name and calculate the mean for latitude and longitude
union_station_coords = combined_stations.groupby('station_name', as_index=False).mean()

print(union_station_coords)

# Updating Station Coordinates with Averaged Values

To ensure consistency and accuracy in our geospatial data, we will update the original latitude and longitude of the start and end stations in our dataset using the averaged coordinates previously calculated.

## Process for Start Stations
We begin by merging the original DataFrame with the averaged coordinates DataFrame on the start station names. This adds the averaged latitude and longitude to each corresponding row.

Next, we update the original start station coordinates with these averaged values. This is done using the `combine_first` method, which fills in the original values only if the averaged value is NaN.

After updating the coordinates, we remove the intermediate columns that were added during the merge, as they are no longer necessary.

## Process for End Stations
We follow a similar process for the end stations, merging the intermediate DataFrame (with updated start station coordinates) with the averaged coordinates DataFrame on the end station names.

Again, we update the original end station coordinates with the averaged values and drop the additional columns added during the merge.

The final DataFrame, `final_df`, now contains the updated coordinates for both start and end stations.


In [ ]:
# Merge for start station latitude and longitude
start_merged = working_df.merge(
    union_station_coords,
    how='left',
    left_on='start_station_name',
    right_on='station_name'
).rename(columns={'avg_latitude': 'start_avg_lat', 'avg_longitude': 'start_avg_lng'})

# Update the start station latitude and longitude
start_merged['start_lat'] = start_merged['start_avg_lat'].combine_first(start_merged['start_lat'])
start_merged['start_lng'] = start_merged['start_avg_lng'].combine_first(start_merged['start_lng'])

# Dropping the additional columns from the first merge
start_merged.drop(['station_name', 'start_avg_lat', 'start_avg_lng'], axis=1, inplace=True)

# Merge for end station latitude and longitude
end_merged = start_merged.merge(
    union_station_coords,
    how='left',
    left_on='end_station_name',
    right_on='station_name',
    suffixes=('', '_end')
).rename(columns={'avg_latitude': 'end_avg_lat', 'avg_longitude': 'end_avg_lng'})

# Update the end station latitude and longitude
end_merged['end_lat'] = end_merged['end_avg_lat'].combine_first(end_merged['end_lat'])
end_merged['end_lng'] = end_merged['end_avg_lng'].combine_first(end_merged['end_lng'])

# Dropping the additional columns from the second merge
final_df = end_merged
final_df.head()

# Selecting Relevant Columns for Model Prediction

To streamline the dataset for the upcoming modeling phase, we now reduce the number of features by selecting only those columns that are relevant for prediction. This step is critical to improve computational efficiency and to avoid having too many features that can actually degrade the performance of many machine learning models. Dropping unrelated features helps to focus the model on the most important information.


In [ ]:
#Dropping the unused columns for the model prediction
columns_to_keep = ['rideable_type',	'started_at',	'start_station_name','end_station_name','ended_at','start_lat',	'start_lng','end_lat','end_lng','member_casual']
finalized_df = final_df[columns_to_keep]
finalized_df.head()

In [ ]:
working_df = finalized_df

# Visualization:  Geospatial Data Distribution

Understanding the distribution of geospatial data in our dataset is crucial for recognizing patterns and potential outliers.

## Latitude Distribution
We first examine the distribution of starting latitudes (`start_lat`). The histogram will show us how often certain latitude ranges occur, which can indicate the most common areas for the start of a ride.

Next, we look at the distribution of ending latitudes (`end_lat`). Comparing this histogram with the starting latitude histogram can reveal whether certain areas are more commonly used as destinations.

## Longitude Distribution
Similarly, we analyze the distribution of starting longitudes (`start_lng`) and ending longitudes (`end_lng`). These histograms will help us to understand the spread of rides longitudinally, indicating the breadth of the area serviced by the rides.

The histograms are plotted with a moderate number of bins (50) to provide a detailed yet comprehensible view of the data distribution. We also include grid lines for easier interpretation of the frequency values.


In [ ]:
import matplotlib.pyplot as plt
4
# Histogram for start_lat
plt.figure(figsize=(10, 5))
plt.hist(working_df['start_lat'], bins=50, color='blue', edgecolor='black')
plt.title('Start Latitude Distribution')
plt.xlabel('Start Latitude')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

# Histogram for start_lng
plt.figure(figsize=(10, 5))
plt.hist(working_df['start_lng'], bins=50, color='green', edgecolor='black')
plt.title('Start Longitude Distribution')
plt.xlabel('Start Longitude')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

# Histogram for end_lat
plt.figure(figsize=(10, 5))
plt.hist(working_df['end_lat'], bins=50, color='red', edgecolor='black')
plt.title('End Latitude Distribution')
plt.xlabel('End Latitude')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

# Histogram for end_lng
plt.figure(figsize=(10, 5))
plt.hist(working_df['end_lng'], bins=50, color='purple', edgecolor='black')
plt.title('End Longitude Distribution')
plt.xlabel('End Longitude')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()


The histograms serve a crucial role in the initial exploratory data analysis, particularly in identifying outliers within the dataset. Outliers can be detected as data points that fall outside the overall pattern of distribution.

To further refine the analysis, especially before deploying machine learning algorithms, standardization of these latitude and longitude values would be beneficial. Standardization (or Z-score normalization) transforms the data to have a mean of zero and a standard deviation of one. By standardizing the data, we improve the robustness and reliability of our models, allowing for better pattern recognition and outlier detection.

# DateTime Conversion and Duration Calculation

In [ ]:
# Convert the 'started_at' and 'ended_at' columns to datetime if they're not already
working_df['started_at'] = pd.to_datetime(working_df['started_at'])
working_df['ended_at'] = pd.to_datetime(working_df['ended_at'])

# Calculate the duration of ride
working_df['ride_duration'] = working_df['ended_at'] - working_df['started_at']

# For duration in seconds
working_df['duration_seconds'] = working_df['ride_duration'].dt.total_seconds()

# Display the first few rows to confirm the new columns
print(working_df[['started_at', 'ended_at', 'ride_duration', 'duration_seconds']].head())
print(working_df['duration_seconds'].describe())

# Visualization: Histogram of Ride Durations

In [ ]:
# Plot a histogram of the 'duration_seconds' column
plt.hist(working_df['duration_seconds'], bins=1000, range=[0, working_df['duration_seconds'].quantile(0.99)])

# Set the title and labels
plt.title('Distribution of Ride Durations')
plt.xlabel('Duration in Seconds')
plt.ylabel('Frequency')

# Show the plot
plt.show()


The histogram illustrates the distribution of ride durations in seconds for a set of bike-sharing data. We can observe a right-skewed distribution, where the majority of rides are short in duration, clustering within the lower second range.

# Enhancing Temporal Resolution in Data

The code snippet focuses on refining the temporal aspects of the dataset. It extracts the precise date and constructs an hourly time interval to facilitate more granular analysis of usage patterns. Additionally, it introduces the 'day_of_week' and 'month' columns to enable trend analysis across different days and months, which could be pivotal for recognizing usage patterns and informing decisions based on temporal factors.

In [ ]:
# Extract the date as YYYY-MM-DD
working_df['date'] = working_df['started_at'].dt.date

# Create time interval by getting just the hour part of the 'started_at' and the next hour
working_df['time_interval'] = working_df['started_at'].dt.strftime('%H') + " - " + \
                              (working_df['started_at'].dt.floor('H') + pd.Timedelta(hours=1)).dt.strftime('%H')

# Ensure 'date' is in datetime format
working_df['date'] = pd.to_datetime(working_df['date'])

# Add a new column 'day_of_week' to get the day name
working_df['day_of_week'] = working_df['date'].dt.day_name()
working_df['month'] = working_df['date'].dt.month_name()

# Now the 'day_of_week' column should contain the name of the day
print(working_df[['date', 'time_interval', 'day_of_week', 'month']].head())


# Visualization: Temporal Patterns in Ride Durations

The provided code is a comprehensive approach to visualizing the temporal patterns in ride durations based on months and days of the week. It uses Seaborn and Matplotlib to create a series of bar plots, depicting both average and total ride durations. The data is first grouped by 'month' and 'day_of_week', with the mean and sum of 'duration_seconds' calculated for each group. The visualizations are designed to reveal trends and insights, such as which months or days are busiest or have the longest rides on average. Custom ordering of the x-axis ensures the data is presented in a logical sequence, from January to December and Monday to Sunday, respectively.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import calendar

# Set the overall aesthetic style of the plots
sns.set(style="whitegrid")

# Order the months from January to December
month_order = [calendar.month_name[i] for i in range(1, 13)]

# Order the weekdays starting from Monday
weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# Assuming 'working_df' is your DataFrame with the 'month' and 'day_of_week' columns

# Group and calculate mean and total durations for both month and day of week
monthly_avg_durations = working_df.groupby('month')['duration_seconds'].mean().reindex(month_order)
monthly_total_durations = working_df.groupby('month')['duration_seconds'].sum().reindex(month_order)
daily_avg_durations = working_df.groupby('day_of_week')['duration_seconds'].mean().reindex(weekday_order)
daily_total_durations = working_df.groupby('day_of_week')['duration_seconds'].sum().reindex(weekday_order)

# Define a function for creating a formatted bar plot
def create_bar_plot(x, y, title, xlabel, ylabel, x_rotation=45, figsize=(12, 6)):
    plt.figure(figsize=figsize)
    sns.barplot(x=x, y=y, order=x)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.xticks(rotation=x_rotation)
    plt.show()

# Plot the average durations for each month
create_bar_plot(monthly_avg_durations.index, monthly_avg_durations.values,
                'Average Ride Duration per Month', 'Month', 'Average Duration in Seconds')

# Plot the total durations for each month
create_bar_plot(monthly_total_durations.index, monthly_total_durations.values,
                'Total Ride Duration per Month', 'Month', 'Total Duration in Seconds')

# Plot the average durations for each day of the week
create_bar_plot(daily_avg_durations.index, daily_avg_durations.values,
                'Average Ride Duration per Day of the Week', 'Day of the Week', 'Average Duration in Seconds')

# Plot the total durations for each day of the week
create_bar_plot(daily_total_durations.index, daily_total_durations.values,
                'Total Ride Duration per Day of the Week', 'Day of the Week', 'Total Duration in Seconds')


In [ ]:
# Assume `columns_to_keep` is a list of column names that you want to keep
columns_to_keep = ['rideable_type','member_casual',	'start_lat','start_lng','start_station_name','end_station_name','date','day_of_week','month','time_interval',	'end_lat','end_lng']  # replace with your actual column names

# Now, select only these columns from `final_df`
working_df = working_df[columns_to_keep]
working_df

In [ ]:
import pandas as pd

# Assuming `df` is your DataFrame and 'date_column' is the name of your date column
working_df['date'] = pd.to_datetime(working_df['date'])

# Now format the dates in 'MM/DD/YYYY' format
working_df['date'] = working_df['date'].dt.strftime('%m/%d/%Y')


In [ ]:
working_df.head()

In [ ]:
weather_filename = '/content/drive/My Drive/IE434 Deep Dive 11 project/Deep Dive 3 (Milestone 2)/weather.csv'

In [ ]:
weather_df = pd.read_csv(weather_filename)
weather_df.head()

Dealt the missing values in the weather data by filling it with zeros and average values (average for temperature)

In [ ]:
# Rename 'DATE' column to 'date' in weather_df
weather_df.rename(columns={'DATE': 'date'}, inplace=True)

# Convert 'DATE' column to datetime
weather_df['date'] = pd.to_datetime(weather_df['date'], format='%m/%d/%Y')

# Now, ensure that the 'date' column in working_df is also in datetime format
working_df['date'] = pd.to_datetime(working_df['date'], format='%m/%d/%Y')


final_df = working_df.merge(weather_df[['date', 'PRCP', 'SNOW', 'TMAX','TMIN']],
                              on='date',
                              how='left')

# Assuming 'df' is your DataFrame and 'column_name' is the name of your column with missing values
final_df['SNOW'] = final_df['SNOW'].fillna(0)

TMAX_column_mean = final_df['TMAX'].mean()
final_df['TMAX'] = final_df['TMAX'].fillna(TMAX_column_mean)

TMIN_column_mean = final_df['TMIN'].mean()
final_df['TMIN'] = final_df['TMIN'].fillna(TMIN_column_mean)

final_df['PRCP'] = final_df['PRCP'].fillna(0)


final_df.head()

In [ ]:
final_df.describe()

In [ ]:
# For all categorical columns
categorical_columns = final_df.select_dtypes(include=['object', 'category']).columns
final_df[categorical_columns].describe()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# For each categorical column
for col in categorical_columns:
    if col == 'start_station_name' or col == 'end_station_name':
      pass
    else:
      sns.countplot(y=col, data=final_df)
      plt.show()


In [ ]:
# Assuming 'df' is your DataFrame
final_df.to_pickle('/content/drive/My Drive/IE434 Deep Dive 11 project/Deep Dive 3 (Milestone 2)/final_df.pkl')
union_station_coords.to_pickle('/content/drive/My Drive/IE434 Deep Dive 11 project/Deep Dive 3 (Milestone 2)/station_coordinates.pkl')


In [ ]:
#loading the above pickle files

final_df_filename = '/content/drive/My Drive/IE434 Deep Dive 11 project/Deep Dive 3 (Milestone 2)/final_df.pkl'
station_coords_df_filename = '/content/drive/My Drive/IE434 Deep Dive 11 project/Deep Dive 3 (Milestone 2)/station_coordinates.pkl'

with open(final_df_filename, 'rb') as file:
    final_df = pickle.load(file)

with open(station_coords_df_filename, 'rb') as file:
    station_coords_df = pickle.load(file)

print(final_df.head())
print(station_coords_df.head())

In [ ]:
#Grouping by Experiments
dropping_columns_for_groupby = ['end_station_name','end_lat','end_lng','member_casual','rideable_type']

new_df = final_df.drop(columns = dropping_columns_for_groupby)
new_df.head()
raw_result=new_df.groupby(['start_lat', 'start_lng','start_station_name','date','day_of_week','month','PRCP','SNOW','TMAX','TMIN']).size().reset_index(name='Daily_Demand')
result = new_df.groupby(['start_lat', 'start_lng','start_station_name','date','day_of_week','month','PRCP','SNOW','TMAX','TMIN']).size().reset_index(name='Daily_Demand')
result.head(20)

The aim is to group by the dataset by date and start_station to calculate the number of rides starting from a particular station. The station information is given by the numerical values of the latitude and longitudes.

In [ ]:
sorted(result['start_lat'].values , reverse=True)

We noticed some outliers (Stations that are very far)in terms of latitude and longitude values and we plan on removing them.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

scaler = MinMaxScaler()
scaler1 =MinMaxScaler()
print(result.shape)
result = result[result['start_lat']<40.755]
print(result.shape)

lat =np.array(result[['start_lat']])
lng =np.array(result[['start_lng']])

result['start_lat'] = scaler.fit_transform(result[['start_lat']])
result['start_lng'] = scaler1.fit_transform(result[['start_lng']])
print(result.shape)
plt.plot(result['start_lat'].values)
plt.show()

plt.plot(result['start_lng'].values)
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt



sns.histplot(result['Daily_Demand'], bins = 50, kde=True)  # 'kde' adds a Kernel Density Estimate plot
plt.title('Distribution of column_name')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.show()

After looking at the distribution, we decided to make this problem a multi label classification problem. We want to split the demand into three categories, low, Medium and Large - Demand

​We converted the numerical Demand variable into Categorical variable (High, Medium, Low) based on​ the frequency distribution curve.​

​
Demand less than 25 percentile (13) is considered as low​ and Demand greater than 75 percentile (41) is considered​ as high demand and the rest as medium demand.

In [ ]:
percentile_25 = result['Daily_Demand'].quantile(0.25)
percentile_75 = result['Daily_Demand'].quantile(0.75)


print(percentile_25)
print(percentile_75)

In [ ]:
import pandas as pd

def categorize_demand(value):
    if value < percentile_25:
        return 'low'
    elif percentile_25 <= value <= percentile_75:
        return 'med'
    else: # value > 41
        return 'large'

# Assuming 'result' is your DataFrame
result['demand_category'] = result['Daily_Demand'].apply(categorize_demand)


In [ ]:
result.drop(columns = ['start_station_name','Daily_Demand','date'],inplace = True)

In [ ]:

# Assuming 'result' is your DataFrame and 'cat_var1', 'cat_var2' are the categorical variables
result = pd.get_dummies(result, columns=['day_of_week', 'month'], drop_first=True)
result.head()


In [ ]:
from sklearn.model_selection import train_test_split

# Assuming 'result' is your DataFrame

# Define the features and the target variable
X = result.drop('demand_category', axis=1)  # features (all columns except 'demand_category')
y = result['demand_category']  # target variable

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

# Now X_train and y_train are your training data, and X_test and y_test are your testing data


In [ ]:
X_train.to_pickle('/content/drive/My Drive/IE434 Deep Dive 11 project/Deep Dive 3 (Milestone 2)/X_train.pkl')
X_test.to_pickle('/content/drive/My Drive/IE434 Deep Dive 11 project/Deep Dive 3 (Milestone 2)/X_test.pkl')

In [ ]:
y_train.to_pickle('/content/drive/My Drive/IE434 Deep Dive 11 project/Deep Dive 3 (Milestone 2)/y_train.pkl')
y_test.to_pickle('/content/drive/My Drive/IE434 Deep Dive 11 project/Deep Dive 3 (Milestone 2)/y_test.pkl')